In [1]:
import chemicals
import chemics as cm
import scipy
import pandas as pd
import pint
from chemicals.iapws import iapws95_Tsat, iapws95_Psat
from sympy import symbols, solve
ureg = pint.UnitRegistry()

In [2]:
def delete_ash_water(fuel):
    print('Eliminando el agua y las cenizas de la composición porcentual del combustible...')
    ash_water = {}
    for element in fuel:
        if element == 'H2O':
            ash_water['H2O'] = fuel['H2O']
        if element == 'Ash':
            ash_water['Ash'] = fuel['Ash']
    new_fuel = fuel.copy()
    try:
        del new_fuel['Ash']
        del new_fuel['H2O']
        del new_fuel['Cl']
    except:
        pass
    correction_factor = (100 - sum(ash_water.values()))/100
    print('Aplicando el factor de corrección...')
    for element in new_fuel:
        new_fuel[element] = fuel[element] / correction_factor
    print('La composición del combustible ahora es:')
    print(new_fuel)
    return new_fuel

In [3]:
def elemental_composition(fuel, elemental_comp = True):
    print('Obteniendo composición molecular...')
    peso_molecular = {}
    print('Transformando composición porcentual a composición elemental a traves del peso molecular...')
    for element in fuel.keys():
        if element == 'Ash':
            peso_molecular[element] = 0
        else:
            if elemental_comp:
                peso_molecular[element] = fuel[element]/cm.mw(element)
            else:
                peso_molecular[element] = fuel[element]
    print(peso_molecular)
    return peso_molecular

In [4]:
def get_reactants_and_products(fuel, relacion_o2n2):
    react = fuel
    sol_s = chemicals.combustion_stoichiometry(fuel)
    print(sol_s)
    prods = {}
    for key in react:
        react[key] =round(react[key], 3)
    for element in sol_s:
        if sol_s[element] < 0:
            react[element] = - round(sol_s[element],3)
        else:
            prods[element] = round(sol_s[element],3)
    react['N2'] = round(relacion_o2n2 * abs(sol_s['O2']), 3)
    prods['N2'] = round(relacion_o2n2 * abs(sol_s['O2']), 3)
    print('Los reactantes son:')
    print(react)
    print('Los productos son:')
    print(prods)
    return react, prods

In [5]:
def get_excess_air(reactantes, productos, lambda_air):
    productos['N2'] = productos['N2'] * lambda_air
    productos['O2'] = reactantes['O2'] * 2 * lambda_air - productos['CO2'] * 2 - productos['H2O']
    reactantes['N2'] = reactantes['N2'] * lambda_air
    reactantes['O2'] = reactantes['O2'] * lambda_air
    print('Con exceso de aire...')
    print('Los reactantes son:')
    print(reactantes)
    print('Los productos son:')
    print(productos)
    return reactantes, productos
#lambda_air = 1.3

In [6]:
def get_fuel(fuel_name, file = 'coal_composition.csv'):
    df = pd.read_csv(file)
    elements = df.columns.tolist()[1:9]
    composition = df[df['fuel_type'] == fuel_name][elements].values.flatten().tolist()
    create_fuel = dict(zip(elements, composition))
    return create_fuel

In [7]:
def get_mass_molecular(react, prods):
    aire = {}
    combustible = {}
    productos_masa = {}
    for element in react:
        if element == 'O2' or element == 'N2':
            aire[element]  = react[element] * cm.mw(element)
        else:
            combustible[element] = react[element] * cm.mw(element)
    for element in prods:
        productos_masa[element] = prods[element] * cm.mw(element)
    productos_masa = {k: v for (k, v) in productos_masa.items() if v >= 0}
    print('La masa de combustible es de: ', (round(sum(combustible.values()),2)),'[kg]')
    print('La masa de aire es de: ', (round(sum(aire.values()),2)),'[kg]')
    print('La masa de los productos es de: ', productos_masa)
    print('La suma de masa de los productos es de: ', (round(sum(productos_masa.values()))),'[kg]')
    suma_reactantes = (round(sum(combustible.values()))) + (round(sum(aire.values())))
    suma_productos = (round(sum(productos_masa.values())))
    diferencia = suma_reactantes - suma_productos
    if diferencia < 1:
        print('Se comprueba la conservación de masa')
    else:
        print('Existe una diferencia de ', diferencia, 'en el balance de masa')
    return combustible, aire, productos_masa

In [8]:
def get_rac(air, fuel, products): #másico o molar
    sum_air = (round(sum(air.values())))
    sum_fuel = (round(sum(fuel.values())))
    rac = sum_air / sum_fuel #kg aire / kg combustible
    gases_humedos = (round(sum(products.values()))) / sum_fuel
    gases_secos = ((round(sum(products.values())))  - products['H2O']) / sum_fuel
    print('La relación Aire / Combustible (RAC) másico es:', rac)
    print('La relación Gases húmedos / Combustible es:', gases_humedos)
    print('La relación Gases secos / Combustible es:', gases_secos)
    return rac, gases_secos, gases_humedos

In [9]:
def epa_factors(fuel_dry, pcs_mjkg):
    pcs_mjkg = pcs_mjkg.to('british_thermal_unit/lb')
    #Dry factor
    fd_list = [fuel_dry['H'], fuel_dry['C'], fuel_dry['S'], fuel_dry['N'], fuel_dry['O']]
    fd_factors = [3.64,1.53,0.57,0.14,-0.46]
    fd = [a * b for a,b in zip(fd_list,fd_factors)]
    fd = sum(fd) / pcs_mjkg.magnitude * ureg.cubic_feet / ureg.british_thermal_unit

    #Wet factor
    fw_list = [fuel_dry['H'], fuel_dry['C'], fuel_dry['S'], fuel_dry['N'], fuel_dry['O'], fuel_wet['H2O']]
    fw_factors = [5.56,1.53,0.575,0.14,-0.46,0.21]
    fw = [a * b for a,b in zip(fw_list,fw_factors)]
    fw = sum(fw) / pcs_mjkg.magnitude * ureg.cubic_feet / ureg.british_thermal_unit

    #Carbon factor
    fc = fuel_dry['C'] * 0.321
    fc = fc / pcs_mjkg.magnitude * ureg.cubic_feet / ureg.british_thermal_unit

    print('El factor fd es:', fd)
    print('El factor fw es:', fw)
    print('El factor fc es:', fc)
    return fd, fw, fc

In [58]:
def get_epa_flows(fuel_mass_wu, pcs_mjkg, lambda_aire, fd, fw, fc):
    pcs_mjkg = pcs_mjkg.to('british_thermal_unit/lb')
    gas_flow_dry = fuel_mass_wu * pcs_mjkg * fd * lambda_aire
    gas_flow_wet = fuel_mass_wu * pcs_mjkg * fw * lambda_aire
    gas_flow_co2 = fuel_mass_wu * pcs_mjkg * fc * lambda_aire
    print('El flujo de gases secos según EPA es:', gas_flow_dry.to('cubic_feet/min'))
    print('El flujo de gases húmedos según EPA es:', gas_flow_wet.to('cubic_feet/min'))
    print('El flujo de gases CO2 según EPA es:', gas_flow_co2.to('cubic_feet/min'))
    return gas_flow_dry, gas_flow_wet, gas_flow_co2

In [149]:
def analytic_flow(masa_productos, masa_combustible, flujo_mass_fuel, rac_dry, rac_wet, tk_op, p_op):
    flujo_mass_fuel = flujo_mass_fuel.magnitude * ureg.kilogram / ureg.hour
    peso_molecular_gases = (cm.mw('O2') * aire_O2 + cm.mw('N2') * aire_N2) / 100
    rho_gas = cm.gas_density.rhog(peso_molecular_gases, p_op.magnitude, tk_op.magnitude) / ureg.meter**3 * ureg.kilogram
    masa_co2 = masa_productos['CO2']
    rho_co2 = cm.gas_density.rhog(cm.mw('CO2'), p_op.magnitude, tk_op.magnitude)/ ureg.meter**3 * ureg.kilogram
    masa_combustible = sum(masa_combustible.values())
    mass_gas_co2 = (masa_co2 / masa_combustible)
    vol_gases_dry = rac_dry / rho_gas * flujo_mass_fuel
    vol_gases_wet = rac_wet / rho_gas * flujo_mass_fuel
    vol_gases_co2 = mass_gas_co2 / rho_co2 * flujo_mass_fuel
    print('El flujo de gases secos análiticos es:', vol_gases_dry.to('cubic_feet/min'))
    print('El flujo de gases húmedos análiticos es:', vol_gases_wet.to('cubic_feet/min'))
    print('El flujo de gases CO2 análiticos es:', vol_gases_co2.to('cubic_feet/min'))
    return vol_gases_dry, vol_gases_wet, vol_gases_co2

In [60]:
def epa_analysis(epa, analytic):
    epa_dry, epa_wet, epa_co2 = [abs(epa[i]- analytic[i]) / epa[i]*100 for i in range(len(epa))]
    print('El porcentaje de error entre el valor análitico y el valor EPA son los siguientes:')
    print('Flujo seco:', round(epa_dry.magnitude,2), '%')
    print('Flujo húmedo:', round(epa_wet.magnitude,2), '%')
    print('Flujo CO2:', round(epa_co2.magnitude,2), '%')
    return epa_dry, epa_wet, epa_co2

In [61]:
def get_temperatura_rocio(productos, presion = 101325):
    presion_parcial = productos['H2O'] / sum(productos.values()) * presion
    print('La presión parcial es:', round(presion_parcial, 3), '[Pa]')
    tsat = iapws95_Tsat(presion_parcial) - 273.15
    print('La temperatura de rocío es: ', round(tsat, 3), '°C')
    return tsat, presion_parcial

In [62]:
def get_masa_condensada(t_operacion, prods, ptotal = 101325):
    pcond = iapws95_Psat(t_operacion + 273.15)
    nh2o = prods['H2O']
    ntotal = sum(prods.values())
    x = symbols('x')
    expr = (nh2o-x)/(ntotal-x) - (pcond/ptotal)
    sol = solve(expr)
    sol = sol[0]
    print('Se condensan', round(sol,3) ,'moles de H2O')
    return sol

In [64]:
info_caldera = pd.read_excel('datos_prueba.xlsx', sheet_name='caldera')
concentracion_co2_df = pd.read_excel('datos_prueba.xlsx', sheet_name='concentraciones_CO2')

In [65]:
#Composición aire
aire_O2 = 21
aire_N2 = 79
razon_N2O2 = aire_N2 / aire_O2

In [151]:
#Datos del problema
concentracion_ppm = 89.44 #mg/m3n
concentracion_co2 = concentracion_co2_df['ppm'].mean()
flujo_masa_combustible = 3000 * ureg.kilogram / ureg.hour #kg/h
PCS = 16.12 * ureg.megajoule/ureg.kilogram #MJ/kg
prod_vapor = 7000 * ureg.kilogram / ureg.hour
p_atm = 101325 * ureg.pascal
t_amb = (20 + 273.15) * ureg.degree_Kelvin
t_gases = (info_caldera['Temperatura Gases [°C]'].mean() + 273.15) * ureg.degree_Kelvin
v_gases = info_caldera['Caudal [m3N/h]'].mean() * ureg.meter_per_second
O2_medido = info_caldera['O2 [%]'].mean()
fuel_wet = {'C':45, 'H':5.5, 'O':43.5, 'S':0.08, 'N':0.09, 'H2O':0, 'Ash':5.83}

In [152]:
lambda_aire = aire_O2 / (aire_O2 - O2_medido)
lambda_aire

1.3665943600867678

In [156]:
#Estequeométrico
fuel_dry = delete_ash_water(fuel_wet)
fuel = elemental_composition(fuel_dry)
react, prods = get_reactants_and_products(fuel, razon_N2O2)
fmass, airemass, productos_mass = get_mass_molecular(react, prods)
rac, rac_seco,rac_humedo = get_rac(airemass, fmass, productos_mass)
temp_rocio, p_parcial = get_temperatura_rocio(prods)
fd, fw, fc = epa_factors(fuel_dry, PCS)
epa = get_epa_flows(flujo_masa_combustible, PCS, 1, fd, fw, fc)
analytic_flow(productos_mass, fmass, flujo_masa_combustible, rac_seco, rac_humedo, t_gases, p_atm)

Eliminando el agua y las cenizas de la composición porcentual del combustible...
Aplicando el factor de corrección...
La composición del combustible ahora es:
{'C': 47.785919082510354, 'H': 5.84050122119571, 'O': 46.193055113093344, 'S': 0.08495274503557397, 'N': 0.09557183816502071}
Obteniendo composición molecular...
Transformando composición porcentual a composición elemental a traves del peso molecular...
{'C': 3.9785129533353056, 'H': 5.7941480369005065, 'O': 2.8872463974681755, 'S': 0.0026498048981776034, 'N': 0.0068231482947826595}
{'CO2': 3.9785129533353056, 'O2': -3.9860765687245223, 'SO2': 0.0026498048981776034, 'N2': 0.0034115741473913297, 'H2O': 2.8970740184502533}
Los reactantes son:
{'C': 3.979, 'H': 5.794, 'O': 2.887, 'S': 0.003, 'N': 0.007, 'O2': 3.986, 'N2': 14.995}
Los productos son:
{'CO2': 3.979, 'SO2': 0.003, 'N2': 14.995, 'H2O': 2.897}
La masa de combustible es de:  100.02 [kg]
La masa de aire es de:  547.61 [kg]
La masa de los productos es de:  {'CO2': 175.111811

(24940.565902446375 <Unit('meter ** 3 / hour')>,
 27125.210925538842 <Unit('meter ** 3 / hour')>,
 4804.634540449095 <Unit('meter ** 3 / hour')>)

In [154]:
#Exceso aire
fuel_dry = delete_ash_water(fuel_wet)
fuel = elemental_composition(fuel_dry)
react, prods = get_reactants_and_products(fuel, razon_N2O2)
react, prods = get_excess_air(react, prods, lambda_aire)
fmass, airemass, productos_mass = get_mass_molecular(react, prods)
rac, rac_seco,rac_humedo = get_rac(airemass, fmass, productos_mass)
temp_rocio, p_parcial = get_temperatura_rocio(prods)
fd, fw, fc = epa_factors(fuel_dry, PCS)
epa = get_epa_flows(flujo_masa_combustible, PCS, lambda_aire, fd, fw, fc)

Eliminando el agua y las cenizas de la composición porcentual del combustible...
Aplicando el factor de corrección...
La composición del combustible ahora es:
{'C': 47.785919082510354, 'H': 5.84050122119571, 'O': 46.193055113093344, 'S': 0.08495274503557397, 'N': 0.09557183816502071}
Obteniendo composición molecular...
Transformando composición porcentual a composición elemental a traves del peso molecular...
{'C': 3.9785129533353056, 'H': 5.7941480369005065, 'O': 2.8872463974681755, 'S': 0.0026498048981776034, 'N': 0.0068231482947826595}
{'CO2': 3.9785129533353056, 'O2': -3.9860765687245223, 'SO2': 0.0026498048981776034, 'N2': 0.0034115741473913297, 'H2O': 2.8970740184502533}
Los reactantes son:
{'C': 3.979, 'H': 5.794, 'O': 2.887, 'S': 0.003, 'N': 0.007, 'O2': 3.986, 'N2': 14.995}
Los productos son:
{'CO2': 3.979, 'SO2': 0.003, 'N2': 14.995, 'H2O': 2.897}
Con exceso de aire...
Los reactantes son:
{'C': 3.979, 'H': 5.794, 'O': 2.887, 'S': 0.003, 'N': 0.007, 'O2': 5.447245119305856, 'N

In [139]:
#Exceso aire
react = {'C': 3.979, 'H': 5.794, 'O': 2.887, 'S': 0.003, 'N': 0.007, 'O2': 5.447245119305856, 'N2': 20.49208242950108}
prods = {'CO2': 3.954, 'SO2': 0.003, 'N2': 20.485, 'H2O': 2.897, 'O2': 1.485 , 'CO': 0.025}
fmass, airemass, productos_mass = get_mass_molecular(react, prods)
rac, rac_seco,rac_humedo = get_rac(airemass, fmass, productos_mass)
temp_rocio, p_parcial = get_temperatura_rocio(prods)
fd, fw, fc = epa_factors(fuel_dry, PCS)
epa = get_epa_flows(flujo_masa_combustible, PCS, lambda_aire, fd, fw, fc)

La masa de combustible es de:  100.02 [kg]
La masa de aire es de:  547.61 [kg]
La masa de los productos es de:  {'CO2': 174.01158600000002, 'SO2': 0.192174, 'N2': 573.8667899999999, 'H2O': 52.189454999999995, 'O2': 47.517030000000005, 'CO': 0.70025}
La suma de masa de los productos es de:  848 [kg]
Se comprueba la conservación de masa
La relación Aire / Combustible (RAC) másico es: 5.48
La relación Gases húmedos / Combustible es: 8.48
La relación Gases secos / Combustible es: 7.958105450000001
La presión parcial es: 10174.998 [Pa]
La temperatura de rocío es:  46.146 °C
El factor fd es: 0.010560052789324452 cubic_foot / british_thermal_unit
El factor fw es: 0.012178179517480178 cubic_foot / british_thermal_unit
El factor fc es: 0.002213348034942754 cubic_foot / british_thermal_unit
El flujo de gases secos según EPA es: 11024.660983674923 cubic_foot / minute
El flujo de gases húmedos según EPA es: 12713.980058346062 cubic_foot / minute
El flujo de gases CO2 según EPA es: 2310.72819529796

In [155]:
epa[1].to('meter**3/h')

21601.18937315595 <Unit('meter ** 3 / hour')>

In [126]:
#Auxiliar para cálcular densidades
rho_gas = cm.gas_density.rhog(cm.mw('NO'), 101325, (20+273.15))/ ureg.meter**3 * ureg.kilogram
rho_gas#.to('lb/cubic_feet').magnitude

1.2473797263138247 <Unit('kilogram / meter ** 3')>

In [127]:
#Auxiliar para pesos moleculares
cm.mw('NO')

30.006

In [70]:
info_caldera['Temperatura Gases [°C]'].mean()

217.43333333333337

In [101]:
#Auxiliar para flujos volumétricos
a = flujo_masa_combustible * PCS.to('british_thermal_units/lb') * fw * lambda_aire
a.to('meter**3/h').magnitude

21601.18937315595